In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from datetime import datetime
from scipy import stats
from collections import defaultdict


In [ ]:

ru_equip_deaths_df = pd.read_csv("./data/russia_losses_equipment.csv")
ru_equip_deaths_df.head()

ru_pers_deaths_df = pd.read_csv("./data/russia_losses_personnel.csv")
ru_pers_deaths_df.head()

global_Ttest = defaultdict(list)

## Data is cumulative.
###

### This data is read from most recent to least recent. 
### for ease of understanding we will reverse this

In [ ]:
ru_equip_deaths_df = ru_equip_deaths_df.sort_index(ascending=False)
ru_equip_deaths_df.reset_index(inplace=True)
ru_equip_deaths_df

ru_pers_deaths_df = ru_pers_deaths_df.sort_index(ascending=False)
ru_pers_deaths_df.reset_index(inplace=True)
ru_pers_deaths_df.head()

In [ ]:
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
# scatter_matrix(ru_equip_deaths_df,figsize=(12,12), diagonal="kde");
# plt.tight_layout()

In [ ]:
ru_equip_deaths_df["tank"].diff(1).plot.line(y="day")
# monthly_mean_tank_loss = 

In [ ]:

ru_pers_deaths_df["personnel"].diff(1).plot.line(y="day")


In [ ]:
 
def check_stationarity(timeseries): 
    # Perform the Dickey-Fuller test 
    result = adfuller(timeseries, autolag='AIC') 
    p_value = result[1] 
    print(f'ADF Statistic: {result[0]}') 
    print(f'p-value: {p_value}') 
    print('Stationary' if p_value < 0.05 else 'Non-Stationary') 

check_stationarity(ru_pers_deaths_df["personnel"].diff(1).dropna())


In [ ]:
ru_pers_deaths_df["personnel"].diff(1).dropna()


In [ ]:
ru_equip_deaths_df["vehicles and fuel tanks"].iloc[0]

In [ ]:
ru_equip_deaths_df.info()
# We know only the first 64 entries have nans. 
for i in range(65):
    if pd.isna(ru_equip_deaths_df["vehicles and fuel tanks"].iloc[i]):
        ru_equip_deaths_df["vehicles and fuel tanks"].iloc[i] = ru_equip_deaths_df["fuel tank"].iloc[i] + ru_equip_deaths_df["military auto"].iloc[i] 

ru_equip_deaths_df.info()


## Create Diff columns

In [ ]:
def create_diff_columns(df_name, column_name):
    """
    Create a new dataframe column for the daily differences in loss rates 
    args: pandas dataframe, str name for column


    """
    df_name[f"{column_name} diff"] = df_name[f"{column_name}"].diff(1)
    df_name[f"{column_name} diff"][0] = df_name[f"{column_name}"][0]

create_diff_columns(ru_equip_deaths_df, "vehicles and fuel tanks")
create_diff_columns(ru_equip_deaths_df, "tank")
create_diff_columns(ru_equip_deaths_df, "APC")
create_diff_columns(ru_equip_deaths_df, "field artillery")
ru_equip_deaths_df
ru_equip_deaths_df["Dt_OBJ"] = ru_equip_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
ru_equip_deaths_df["Int_month"] = ru_equip_deaths_df["Dt_OBJ"].dt.month
ru_equip_deaths_df["Int_year"] = ru_equip_deaths_df["Dt_OBJ"].dt.year
ru_pers_deaths_df["Dt_OBJ"] = ru_pers_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
ru_pers_deaths_df["Int_month"] = ru_pers_deaths_df["Dt_OBJ"].dt.month
ru_pers_deaths_df["Int_year"] = ru_pers_deaths_df["Dt_OBJ"].dt.year

for i in range(len(ru_equip_deaths_df)):
        if ru_equip_deaths_df["Int_month"].iloc[i] == 12:
            ru_equip_deaths_df.iloc[i,-1] += 1 

In [ ]:
ru_equip_deaths_df["field artillery diff"].plot.line(y="day")

## Groupby months to evaluate each monthly trend

#### We'll use this to guide our seasonality checks

In [ ]:
ru_equip_deaths_df["Dt_OBJ"] = ru_equip_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

# ru_equip_deaths_df.groupby(pd.Grouper(key='date', freq='M'))

type(ru_equip_deaths_df["Dt_OBJ"])
# ru_equip_deaths_df["Dt_OBJ"]
# ru_equip_deaths_df.head()
def monthly_analysis(df_obj, c_name):
    by_month_df = ru_equip_deaths_df.groupby(pd.Grouper(key='Dt_OBJ', freq='M'))["tank diff"]
    




ru_by_month_df = ru_equip_deaths_df.groupby(pd.Grouper(key='Dt_OBJ', freq='M'))
ru_by_month_df.sum()["tank diff"].plot.line()
ru_by_month_df.sum()["tank diff"]


ru_pers_deaths_df["pers diff"] = ru_pers_deaths_df["personnel"].diff(1)
ru_pers_deaths_df["pers diff"][0] = 2800
# ru_pers_deaths_df["pers diff"]

ru_pers_deaths_df["Dt_OBJ"] = ru_pers_deaths_df['date'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
ru_pers_deaths_df.head()
ru_pers_by_month_df = ru_pers_deaths_df.groupby(pd.Grouper(key='Dt_OBJ', freq='M'))
ru_pers_by_month_df.sum()["pers diff"]
ru_pers_deaths_df['Dt_OBJ'][1].month


# Create season dataframes

In [ ]:
print(ru_pers_deaths_df['Dt_OBJ'][0].month)


def create_seasonal_comparisons(df_name, column_name, color):
    df_name["Int_month"] = df_name["Dt_OBJ"].dt.month
    ru_deaths_summerdf = df_name[(df_name["Int_month"] == 6) | (df_name["Int_month"] == 7) |
                                                (df_name["Int_month"] == 8)]
    ru_deaths_falldf = df_name[(df_name["Int_month"] == 9) | (df_name["Int_month"] == 10) |
                                                (df_name["Int_month"] == 11)]
    ru_deaths_winterdf = df_name[(df_name["Int_month"] == 12) | (df_name["Int_month"] == 1) |
                                                (df_name["Int_month"] == 2)]
    ru_deaths_springdf = df_name[(df_name["Int_month"] == 3) | (df_name["Int_month"] == 4) |
                                                (df_name["Int_month"] == 5)]
    # Add Nones between years
    ru_deaths_summerdf[f"{column_name}"].plot()
    ru_deaths_springdf[f"{column_name}"].plot()
    ru_deaths_falldf[f"{column_name}"].plot()
    ru_deaths_winterdf[f"{column_name}"].plot()
    print("Summer mean:", ru_deaths_summerdf[f"{column_name}"].mean())
    print("Spring mean:" ,ru_deaths_springdf[f"{column_name}"].mean())
    print("Fall mean:" ,ru_deaths_falldf[f"{column_name}"].mean())
    print("Winter mean:" ,ru_deaths_winterdf[f"{column_name}"].mean())
    
    print("Winter vs Spring:" , stats.ttest_ind(ru_deaths_winterdf[f"{column_name}"], ru_deaths_springdf[f"{column_name}"]))
    print("Summer vs Spring:" ,stats.ttest_ind(ru_deaths_summerdf[f"{column_name}"], ru_deaths_springdf[f"{column_name}"]))
    print("Fall vs Spring:" ,stats.ttest_ind(ru_deaths_falldf[f"{column_name}"], ru_deaths_springdf[f"{column_name}"]))

# Add color vairable, change seasonal plot


def trend_line_w_outliers(df_name, column_name, draw_data=False, draw_trend=True):
    
    
    ru_deaths_summerdf = df_name[(df_name["Int_month"] == 6) | (df_name["Int_month"] == 7) |
                                                (df_name["Int_month"] == 8)]
    ru_deaths_falldf = df_name[(df_name["Int_month"] == 9) | (df_name["Int_month"] == 10) |
                                                (df_name["Int_month"] == 11)]
    ru_deaths_winterdf = df_name[(df_name["Int_month"] == 12) | (df_name["Int_month"] == 1) |
                                                (df_name["Int_month"] == 2)]
    ru_deaths_springdf = df_name[(df_name["Int_month"] == 3) | (df_name["Int_month"] == 4) |
                                                (df_name["Int_month"] == 5)]
    plt.figure(figsize=(12, 6))

    summer_std = ru_deaths_summerdf[f"{column_name}"].std()
    fall_std = ru_deaths_falldf[f"{column_name}"].std()
    winter_std = ru_deaths_winterdf[f"{column_name}"].std()
    spring_std = ru_deaths_springdf[f"{column_name}"].std()
    if draw_data:
        plt.plot(df_name["Dt_OBJ"],df_name[f"{column_name}"])
    
    draw_trend_line(ru_deaths_summerdf,column_name,draw_trend)
    draw_trend_line(ru_deaths_falldf,column_name,draw_trend)
    draw_trend_line(ru_deaths_winterdf,column_name,draw_trend)
    draw_trend_line(ru_deaths_springdf,column_name,draw_trend)
    
    

def draw_trend_line(df_name, column_name, draw_trend=True):
    #Attach december to the next year for plotting purposes
    


    yearly_data = df_name.groupby(df_name["Int_year"])
    df_name_mean = df_name[f"{column_name}"].median()
    
    for year, data in yearly_data:
        # print("yearly", year, data.head())
        df_name_std = data[f"{column_name}"].std()
        df_name_mean = data[f"{column_name}"].mean()
        # print(f"{year}:STD : {df_name_std}")
        # print(f"{year}:mean : {df_name_mean}")
        
        first_point = data.iloc[0]
        last_point = data.iloc[-1]
        if draw_trend:
            plt.plot([first_point["Dt_OBJ"], last_point["Dt_OBJ"]], 
                    [first_point[f'{column_name}'], last_point[f'{column_name}']], 
                    marker='o')
        
        for points in range(len(data)):
            
            if (data[f"{column_name}"].iloc[points] > (df_name_std * 2) + df_name_mean):
                #or (df_name[f"{column_name}"].iloc[points] < df_name_mean - df_name_std)
                # print(" Outlier at ",data["Int_month"].iloc[points],  data[f"{column_name}"].iloc[points])

                plt.plot(data["Dt_OBJ"].iloc[points],data[f"{column_name}"].iloc[points], marker='x',color="black")

    
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('Trend Lines for Each Month')
    plt.legend()
    plt.grid(True)

# y = df['passengers']
# x = df['lag_1']
# model = sm.OLS(y, sm.add_constant(x))
# results = model.fit()
# b, m = results.params
# IQR, standard deviation to find outliers. plot them, take linear regression lines for each season

In [ ]:
for i in range(len(ru_pers_deaths_df)):
        if ru_pers_deaths_df["Int_month"].iloc[i] == 12:
            ru_pers_deaths_df.iloc[i, ru_pers_deaths_df.columns.get_loc('Int_year')] += 1

trend_line_w_outliers(ru_pers_deaths_df, "pers diff",draw_data=True)


# SNS.REGPLOT

In [ ]:


ru_equip_deaths_df[ru_equip_deaths_df["Int_month"] == 12]
#remember iterrows

In [ ]:
trend_line_w_outliers(ru_equip_deaths_df, "tank diff",draw_data=False,draw_trend=True)
# Maybe remove initial

In [ ]:
trend_line_w_outliers(ru_equip_deaths_df, "APC diff",draw_data=True,draw_trend=False)

In [ ]:
trend_line_w_outliers(ru_equip_deaths_df, "field artillery diff",draw_data=True,draw_trend=False)

In [ ]:
plt.figure(figsize=(12, 6))
ru_pers_deaths_df.plot.line(x="Dt_OBJ", y="pers diff")

In [ ]:

create_seasonal_comparisons(ru_pers_deaths_df, "pers diff")


In [ ]:
create_seasonal_comparisons(ru_equip_deaths_df, "APC diff")

In [ ]:
create_seasonal_comparisons(ru_equip_deaths_df, "field artillery diff")

In [ ]:

create_seasonal_comparisons(ru_equip_deaths_df, "vehicles and fuel tanks diff")

In [ ]:
def seasonal_decomposition(df_name, column_name):
    equip_analysis = df_name.loc[0::,["Dt_OBJ", f"{column_name} diff"]]
    equip_analysis.set_index("Dt_OBJ", inplace=True)
    decompose_result = seasonal_decompose(equip_analysis,period=90)
    trend = decompose_result.trend
    seasonal = decompose_result.seasonal
    resid_season = decompose_result.resid
    decompose_result.plot()
    return decompose_result

APC_decomp = seasonal_decomposition(ru_equip_deaths_df,"APC")

In [ ]:
trend.plot()
plt.axvline(x=datetime(2022,8,29),color="red")
plt.axvline(x=datetime(2023,9,1))
plt.axvline(x=datetime(2022,7,3),color="black")

In [ ]:
pers_decomp = seasonal_decomposition(ru_pers_deaths_df,"pers")

In [ ]:
trend.plot()
plt.axvline(x=datetime(2022,8,29),color="red")
plt.axvline(x=datetime(2023,9,1))
plt.axvline(x=datetime(2022,7,3),color="black")

In [ ]:
ru_by_month_df["tank diff"].mean().plot.line()


In [ ]:
ru_by_month_df.sum()["tank diff"].plot.line()

In [ ]:
ru_pers_by_month_df.sum()["pers diff"]
ru_pers_by_month_df["pers diff"].mean().plot.line()

In [ ]:
#testing lags at different areas

# group into seasons, compare means of seasons, drop back into days, t-test
# seasonal decomposition sarima

# outlier analysis

### Checks for stationarity

In [ ]:
check_stationarity(ru_equip_deaths_df["vehicles and fuel tanks diff"])

In [ ]:
ru_equip_deaths_df["vehicles and fuel tanks"].diff(1).dropna().plot.line(y="day")

In [ ]:
check_stationarity(ru_equip_deaths_df["tank"].diff(1).dropna())

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(ru_equip_deaths_df["tank"].diff(1).dropna())
autocorrelation_plot(ru_equip_deaths_df["tank"].dropna())


In [ ]:
autocorrelation_plot(ru_equip_deaths_df["vehicles and fuel tanks"].diff(1).dropna())

In [ ]:
ru_equip_deaths_df["tank"].diff(1).dropna().hist()

In [ ]:
ru_pers_deaths_df["personnel"].diff(-1).dropna().hist()

In [ ]:
autocorrelation_plot(ru_pers_deaths_df["personnel"].diff(-1).dropna())

In [ ]:
plot_acf(ru_equip_deaths_df["tank"].diff(1).dropna()) 
plot_pacf(ru_equip_deaths_df["tank"].diff(1).dropna()) 
plt.show()

In [ ]:
autocorrelation_plot(ru_equip_deaths_df["tank"].diff(1).dropna())

# Linear regression for the Seasons

###### For everything turn, turn, turn

In [ ]:
# use pd.cut to bin by seasons (i.e. spring, summer, fall, winter), according to the ukrainian climate.
# hypothesis, there will be a statistically significant change in casualty rates 
# across vehicle and personnel in spring and winter.
# Furthermore that change will be lower than summer and fall casualty rates

# The SARIMA MODELS

## Code below here is Fitting and reviewing models.
##### These may be computationally expensive!

In [ ]:
  
# Define SARIMA parameters 
p, d, q = 8, 1, 3 
P, D, Q, s = 1, 1, 1, 90   # Assuming tri-month seasonality, fit to seasons
  
# Fit the SARIMA model 
# ar_model = ARIMA(ru_equip_deaths_df["tank"].diff(1).dropna(), order=(p,d,q))
model = SARIMAX(ru_equip_deaths_df["tank"].diff(1).dropna(), order=(p, d, q), seasonal_order=(P, D, Q, s)) 
results = model.fit() 


In [ ]:
'''P VALUE TESTER
'''

p, d, q = 9, 1, 1 
P, D, Q, s = 1, 1, 1, 90   # just goofin around
  
# Fit the SARIMA model 
# ar_model = ARIMA(ru_equip_deaths_df["tank"].diff(1).dropna(), order=(p,d,q))
model_p_fault = SARIMAX(ru_equip_deaths_df["tank"].diff(1).dropna(), order=(p, d, q), seasonal_order=(P, D, Q, s)) 
results_p_fault = model_p_fault.fit() 

In [ ]:
p, d, q = 9, 1, 1 
P, D, Q, s = 1, 1, 1, 30   # Assuming monthly seasonality 
  
# Fit the SARIMA model 
# ar_model = ARIMA(ru_equip_deaths_df["tank"].diff(1).dropna(), order=(p,d,q))
model_third = SARIMAX(ru_equip_deaths_df["tank"].diff(1).dropna(), order=(p, d, q), seasonal_order=(P, D, Q, s)) 
results_third = model_third.fit() 

In [ ]:
predictions = results.predict(start=0, end=len(ru_equip_deaths_df)-1)
predictions_third = results_third.predict(start=0, end=len(ru_equip_deaths_df)-1)


In [ ]:
forecast = results.get_forecast(steps=90)

forecast_df = pd.DataFrame(forecast.predicted_mean)
forecast_df

In [ ]:
results.summary()

In [ ]:

'''ARIMA
Model:	SARIMAX(1, 1, 3)x(1, 1, [1], 90)	Log Likelihood	-2680.787
Date:	Fri, 19 Jul 2024	AIC	5375.575
Time:	15:48:34	BIC	5408.181
Sample:	0	HQIC	5388.116

'''
results_p_fault.summary()

In [ ]:
results_third.summary()
#aic: 5673
'''ARIMA
Model:	SARIMAX(9, 1, 1)x(1, 1, 1, 30)	Log Likelihood	-2821.540
                    AIC 5669.081
Time:	15:00:55	BIC	5730.599
Sample:	0	HQIC	5692.660




'''

In [ ]:
# fig, ax = plt.subplots()
forecast_third = results_third.get_forecast(steps=90)

forecast_df_third = pd.DataFrame(forecast_third.predicted_mean)
forecast_df.plot.line()
# forecast_df_third.plot.line()
ru_equip_deaths_df["tank"].diff(1).dropna().plot.line()

#### Tank losses per day cannot be negative
##### Roll through predictions and set negative predicts to zero

In [ ]:
predictions[predictions < 0] = 0


In [ ]:
autocorrelation_plot(predictions.dropna())
# autocorrelation_plot(predictions_third.dropna())
autocorrelation_plot(ru_equip_deaths_df["tank"].diff(1).dropna())

In [ ]:
ru_equip_deaths_df["tank"].diff(1).dropna().plot.line()
predictions.plot.line()


In [ ]:
residuals = ru_equip_deaths_df["tank"].diff(1).dropna() - predictions
residuals.dropna(inplace=True)

residuals = pd.DataFrame(residuals)
residuals.reset_index()
residuals.columns = ['diff']


fig, ax = plt.subplots(figsize=(14, 4))
# ax.scatter(range(len(predictions)), predictions, color="red", label="y true values")
ax.scatter(range(len(residuals)), residuals, color="k", label='y_hat')
# for i in range(1, len(predictions)):
#     plt.vlines(i,residuals['diff'][i],predictions[i],color="k", linestyle="--")

    # 0,y_hat[0],y[0],color="k", linestyle="--"
ax.legend()
plt.title("AVG EXP")
plt.show()


# AIC, BIC

In [ ]:
cas_residuals = results.resid
cas_residuals
predictions[predictions < 0] = 0
plt.scatter(predictions[1::], cas_residuals)
plt.axhline(y=0,color="orange",linestyle="--")
plt.show()


In [ ]:
autocorrelation_plot(cas_residuals)

In [ ]:
ru_equip_deaths_df["tank"].diff(1).plot.line(y="day")

In [ ]:
ru_equip_deaths_df["tank"].plot.line(y="day")
ru_equip_deaths_df["tank"]